In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import urllib.parse
from scipy.stats import linregress
from api_keys import omdb_api_key
from pprint import pprint

#print(omdb_api_key)

# Sets the path to our movie csv file
path = 'Resources/boxoffice.csv'

In [2]:
# Pulls in a list of ~16000 top movies of all time
top_boxoffice_df = pd.read_csv(path)

# Drops the maunally created index column
top_boxoffice_df.drop(labels=['index'], axis=1, inplace=True)

top_boxoffice_df

,rank,title,studio,lifetime_gross,year
0,1,Star Wars: Episode VII,BV,936662225,2015
1,2,Avengers: Endgame,BV,857190335,2019
2,3,Avatar,Fox,760507625,2009
3,4,Black Panther,BV,700059566,2018
4,5,Avengers: Infinity War,BV,678815482,2018
...,...,...,...,...,...
16537,16538,Dog Eat Dog,IFC,80,2009
16538,16539,Paranoid Girls,NaN,78,2015
16539,16540,Confession of a Child of the Century,Cohen,74,2015
16540,16541,Storage 24,Magn.,72,2013


In [3]:
top1k_boxoffice_df = pd.read_csv(path, nrows = 1000)

# Drops the maunally created index column
top1k_boxoffice_df.drop(labels=['index'], axis=1, inplace=True)

top1k_boxoffice_df

,rank,title,studio,lifetime_gross,year
0,1,Star Wars: Episode VII,BV,936662225,2015
1,2,Avengers: Endgame,BV,857190335,2019
2,3,Avatar,Fox,760507625,2009
3,4,Black Panther,BV,700059566,2018
4,5,Avengers: Infinity War,BV,678815482,2018
...,...,...,...,...,...
995,996,City of Angels,WB,78685114,1998
996,997,The English Patient,Mira.,78676425,1996
997,998,Teenage Mutant Ninja Turtles II,NL,78656813,1991
998,999,A.I. Artificial Intelligence,WB,78616689,2001


In [4]:
bot1k_boxoffice_df = pd.read_csv(path, skiprows= 15542, nrows = 1001, names=['rank', 'index', 'title', 'studio', 'lifetime_gross', 'year'])

# Drops the maunally created index column
bot1k_boxoffice_df.drop(labels=['index'], axis=1, inplace=True)

bot1k_boxoffice_df

,rank,title,studio,lifetime_gross,year
0,15541,December,IRS,5868,1991
1,15542,Buzzkill,Indic.,5860,2012
2,15543,Gaudi Afternoon,Cn4,5858,2003
3,15544,Top Spin,FRun,5858,2015
4,15545,The Work,Orch.,5853,2017
...,...,...,...,...,...
996,16537,Dog Eat Dog,IFC,80,2009
997,16538,Paranoid Girls,NaN,78,2015
998,16539,Confession of a Child of the Century,Cohen,74,2015
999,16540,Storage 24,Magn.,72,2013


In [5]:
# Creates a list of all of the movie titles in the top 1k movies from the dataframe
movies_list = top1k_boxoffice_df['title'].tolist()

# Creates empty lists for all of the information we want from the json requests
req_imdbID = []
req_titles = []
req_studio = []
req_boxoffice = []
req_runtime = []
req_age_rating = []
req_release_date = []
req_country = []
req_genre = []
req_imdb_rating = []
req_imdb_votecount = []
req_critic_rating = []
req_directors = []

print('Beginning OMDb data retriveal:')
print('______________________________')

try:
    
    # Loops through the dataframe and pulls the rest of  the information for the movies from OMDb API
    for movie_title in movies_list:

        # Encodes the entire movie title to account for characters
        encoded_title = urllib.parse.quote(movie_title)

        # Sets the base url for OMDb API and prints it out, comment out the printing function if using in public
        omdb_url = f'http://www.omdbapi.com/?t={encoded_title}&apikey={omdb_api_key}'
        print(f"API Request URL: {omdb_url}")

        # Creates a request to the OMDb API
        response = requests.get(omdb_url)
        request = response.json()

        # Tells us what is wrong with a request if it is not good
        if response.status_code != 200 or request['Response'] == 'False':
            print(f"Failed to retrieve movie '{movie_title}' from OMDb API.")
            print(f"Response: {request}")
            continue

        # Skips tv series from the data set
        if request['Type'] == 'series':
            print(f"'{movie_title}' is a TV series. Skipping...")
            continue

        # Gets the studio of each movie when it loops through the list
        studio_info = top1k_boxoffice_df.loc[top1k_boxoffice_df['title'] == movie_title, 'studio'].values[0]

        # Adds the information of each title to each list
        req_imdbID.append(request['imdbID'])
        req_titles.append(request['Title'])
        req_studio.append(studio_info)
        req_boxoffice.append(request['BoxOffice'])
        req_runtime.append(request['Runtime'])
        req_age_rating.append(request['Rated'])
        req_release_date.append(request['Released'])
        req_country.append(request['Country'])
        req_genre.append(request['Genre'])
        req_imdb_rating.append(request['imdbRating'])
        req_imdb_votecount.append(request['imdbVotes'])
        req_critic_rating.append(request['Metascore'])
        req_directors.append(request['Director'])

# Prints any errors that occured
except Exception as e:    
    print('An error occurred:', str(e))

print('______________________________')
print('Data retrieval complete')

    

Beginning OMDb data retriveal:
______________________________
API Request URL: http://www.omdbapi.com/?t=Star%20Wars%3A%20Episode%20VII&apikey=e82a912b
API Request URL: http://www.omdbapi.com/?t=Avengers%3A%20Endgame&apikey=e82a912b
API Request URL: http://www.omdbapi.com/?t=Avatar&apikey=e82a912b
API Request URL: http://www.omdbapi.com/?t=Black%20Panther&apikey=e82a912b
API Request URL: http://www.omdbapi.com/?t=Avengers%3A%20Infinity%20War&apikey=e82a912b
API Request URL: http://www.omdbapi.com/?t=Titanic&apikey=e82a912b
API Request URL: http://www.omdbapi.com/?t=Jurassic%20World&apikey=e82a912b
API Request URL: http://www.omdbapi.com/?t=The%20Avengers&apikey=e82a912b
API Request URL: http://www.omdbapi.com/?t=Star%20Wars%3A%20The%20Last%20Jedi&apikey=e82a912b
API Request URL: http://www.omdbapi.com/?t=Incredibles%202&apikey=e82a912b
API Request URL: http://www.omdbapi.com/?t=The%20Dark%20Knight&apikey=e82a912b
API Request URL: http://www.omdbapi.com/?t=Rogue%20One%3A%20A%20Star%20Wa

In [6]:
# Creates our usable dataframe for our movies
total_movie_df = pd.DataFrame({
    'IMDb ID': req_imdbID,
    'Movie Title': req_titles,
    'Studio': req_studio,
    'Box Office Sales': req_boxoffice,
    'Movie Runtime': req_runtime,
    'Age Rating': req_age_rating,
    'Release Date' : req_release_date,
    'Country' : req_country,
    'Genre': req_genre,
    'IMDb Rating' : req_imdb_rating,
    'IMDb Vote Count' : req_imdb_votecount,
    'Critic Rating' : req_critic_rating,
    'Director(s)' : req_directors,
})


total_movie_df

,IMDb ID,Movie Title,Studio,Box Office Sales,Movie Runtime,Age Rating,Release Date,Country,Genre,IMDb Rating,IMDb Vote Count,Critic Rating,Director(s)
0,tt2488496,Star Wars: Episode VII - The Force Awakens,BV,"$936,662,225",138 min,PG-13,18 Dec 2015,United States,"Action, Adventure, Sci-Fi",7.8,"945,099",80,J.J. Abrams
1,tt4154796,Avengers: Endgame,BV,"$858,373,000",181 min,PG-13,26 Apr 2019,United States,"Action, Adventure, Drama",8.4,"1,172,062",78,"Anthony Russo, Joe Russo"
2,tt0499549,Avatar,Fox,"$785,221,649",162 min,PG-13,18 Dec 2009,United States,"Action, Adventure, Fantasy",7.9,"1,338,789",83,James Cameron
3,tt1825683,Black Panther,BV,"$700,426,566",134 min,PG-13,16 Feb 2018,United States,"Action, Adventure, Sci-Fi",7.3,"798,824",88,Ryan Coogler
4,tt4154756,Avengers: Infinity War,BV,"$678,815,482",149 min,PG-13,27 Apr 2018,United States,"Action, Adventure, Sci-Fi",8.4,"1,116,082",68,"Anthony Russo, Joe Russo"
...,...,...,...,...,...,...,...,...,...,...,...,...,...
967,tt0120632,City of Angels,WB,"$78,685,114",114 min,PG-13,10 Apr 1998,"United States, Germany","Drama, Fantasy, Romance",6.7,"123,806",54,Brad Silberling
968,tt0116209,The English Patient,Mira.,"$78,676,425",162 min,R,06 Dec 1996,"United States, United Kingdom","Drama, Romance, War",7.4,"194,715",87,Anthony Minghella
969,tt0103060,Teenage Mutant Ninja Turtles II: The Secret of...,NL,"$78,656,813",88 min,PG,22 Mar 1991,"Hong Kong, United States","Action, Adventure, Comedy",6.0,"57,832",45,Michael Pressman
970,tt0212720,A.I. Artificial Intelligence,WB,"$78,616,689",146 min,PG-13,29 Jun 2001,"United States, United Kingdom","Drama, Sci-Fi",7.2,"314,145",65,Steven Spielberg
